In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Conv2D, LSTM, CuDNNLSTM, Flatten, Dropout, Input, TimeDistributed, Reshape
from keras import optimizers
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
class trainerHelper():
    
    def __init__(self, data, labels, win_size, num_sensor_data, num_classes):
        self.data = data
        self.labels = labels
        self.win_size = win_size
        self.sensor_data = num_sensor_data
        self.num_classes = num_classes
        self.data_length = self.data.shape[0]
        print(self.data_length)
        self.start = 0
        
    def windows(self):
 
        while self.start + self.win_size < self.data_length:
            yield int(self.start), int(self.start + self.win_size)
            self.start += (self.win_size/2)
    
    def segment_data(self):
        self.start = 0
        segments = np.empty((0, self.win_size, self.sensor_data))
        labels= np.empty((0, self.num_classes))
        for (start, end) in self.windows():
            x = np.zeros((1, self.win_size, self.sensor_data))
            x[0,:] = self.data[start:end,:]
            label = self.labels[start:end,:]
            if(x.shape[1] == self.win_size):
                segments = np.vstack([segments,x])
                lb = np.zeros((1, self.num_classes))
                lb[0,:] = label[-1]
                labels = np.vstack([labels,lb])
        return segments, labels

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

def prepare_data(train_data, test_data):
    encoder = OneHotEncoder()
    train_labels = encoder.fit_transform(train_data['labels'].values.reshape(-1,1)).toarray()
    test_labels = encoder.transform(test_data['labels'].values.reshape(-1,1)).toarray()
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_data.drop(['labels'], axis=1, inplace=True)
    test_data.drop(['labels'], axis=1, inplace=True)
    train_data = scaler.fit_transform(train_data)
    test_data = scaler.transform(test_data)
    
    return train_data, test_data, train_labels, test_labels

In [4]:
adl_1 = pd.read_csv("ADL1Opportunity_taskB2_S1.csv",header=None)
adl_2 = pd.read_csv("ADL2Opportunity_taskB2_S1.csv",header=None)
adl_3 = pd.read_csv("ADL3Opportunity_taskB2_S1.csv",header=None)
adl_4 = pd.read_csv("ADL4Opportunity_taskB2_S1.csv",header=None)
adl_5 = pd.read_csv("ADL5Opportunity_taskB2_S1.csv",header=None)
drill = pd.read_csv("Drill1Opportunity_taskB2.csv",header=None)

In [5]:
train_frames = [adl_1, adl_2, adl_3, drill]
test_frames = [adl_4, adl_5]
train_data = pd.concat(train_frames)
test_data = pd.concat(test_frames)
train_data.rename(columns ={113: 'labels'}, inplace =True)
test_data.rename(columns ={113: 'labels'}, inplace =True)

In [6]:
print('Shape of train data is {}'.format(train_data.shape))
print('Shape of test data is {}'.format(test_data.shape))
train_data.head()

Shape of train data is (171500, 114)
Shape of test data is (63083, 114)


,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,labels
0,148.0,956.0,-358.0,19.0,986.0,196.0,95.0,975.0,152.0,194.0,...,319,-845,-20,57,42,57,20,42,175,1
1,89.0,973.0,-287.0,10.0,1004.0,162.0,125.0,968.0,122.0,224.0,...,325,-847,-17,38,31,38,17,31,175,1
2,130.0,988.0,-418.0,-11.0,1014.0,202.0,127.0,1002.0,113.0,279.0,...,328,-852,27,31,15,31,-27,15,175,1
3,89.0,980.0,-425.0,-47.0,1025.0,191.0,110.0,1006.0,105.0,353.0,...,321,-852,26,22,-2,22,-26,-2,175,1
4,64.0,857.0,-391.0,-8.0,1022.0,204.0,97.0,1002.0,93.0,548.0,...,321,-850,22,45,-7,45,-22,-7,175,1


In [7]:
scaled_train, scaled_test, train_labels, test_labels = prepare_data(train_data, test_data)

In [9]:
print(scaled_train.shape)
print(train_labels)

(171500, 113)
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]


In [325]:
print(scaled_test)

[[ 0.451319    0.55016061  0.55583438 ...,  0.41258607  0.3406629
   0.80560748]
 [ 0.45576862  0.55003706  0.55795773 ...,  0.41090665  0.34028023
   0.80560748]
 [ 0.45672211  0.54954287  0.55679954 ...,  0.41160241  0.34133993
   0.80560748]
 ..., 
 [ 0.46540947  0.41919941  0.55583438 ...,  0.41102661  0.34142824
   0.54392523]
 [ 0.46540947  0.41919941  0.55583438 ...,  0.41177035  0.34148711
   0.54766355]
 [ 0.46540947  0.41919941  0.55583438 ...,  0.41088266  0.34142824
   0.5046729 ]]


In [331]:
win_size = 15
classes = 18
num_sensors = scaled_train.shape[1]

train_segments = np.empty((0, win_size, num_sensors))
train_segments_labels= np.empty((0, classes))
last_idx = 0
for adl in train_frames:

    trainer_helper = trainerHelper(scaled_train[last_idx:last_idx+adl.shape[0]], train_labels[last_idx:last_idx+adl.shape[0]], win_size, num_sensors, classes)
    segments, labels = trainer_helper.segment_data()
    train_segments = np.vstack([train_segments, segments])
    train_segments_labels = np.vstack([train_segments_labels, labels])
    last_idx = adl.shape[0]

print('Train data has been segmented')

test_segments = np.empty((0, win_size, num_sensors))
test_segments_labels= np.empty((0, classes))
last_idx = 0
for adl in test_frames:

    trainer_helper = trainerHelper(scaled_test[last_idx:last_idx+adl.shape[0]], test_labels[last_idx:last_idx+adl.shape[0]], win_size, num_sensors, classes)
    segments, labels = trainer_helper.segment_data()
    test_segments = np.vstack([test_segments, segments])
    test_segments_labels = np.vstack([test_segments_labels, labels])
    last_idx = adl.shape[0]
    
print('Test data has been segmented')    

51088
32223
33274
54915
Train data has been segmented
32955
30128
Test data has been segmented


In [332]:
train_segments.shape

(22860, 15, 113)

In [333]:
train_segments_labels.shape

(22860, 18)

In [334]:
# reshape input for CNN
reshaped_train = train_segments.reshape(train_segments.shape[0], win_size, num_sensors, 1)
reshaped_test = test_segments.reshape(test_segments.shape[0], win_size, num_sensors, 1)

In [394]:
model = Sequential()
size_of_kernel = (3,1)
num_filters = 64
num_lstm_cells = 128
dropout_prob = 0.5
inputshape = (win_size, num_sensors, 1)
batchshape = (None, win_size, num_sensors)

In [395]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=1,
                 activation='relu', input_shape=inputshape, name='1_conv_layer'))

In [396]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=1,
                 activation='relu', name='2_conv_layer'))

In [397]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=1,
                 activation='relu', name='3_conv_layer'))

In [398]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=1,
                 activation='relu', name='4_conv_layer'))

In [399]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=1,
                 activation='relu', name='5_conv_layer'))

In [400]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=1,
                 activation='relu', name='6_conv_layer'))

In [401]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=1,
                 activation='relu', name='7_conv_layer'))

In [402]:
model.add(Reshape((-1, num_filters*num_sensors)))

In [403]:
model.add(CuDNNLSTM(num_lstm_cells, return_sequences=True, name='1_lstm_layer'))

model.add(Dropout(dropout_prob, name='2_dropout_layer'))

model.add(CuDNNLSTM(num_lstm_cells, return_sequences=False, name='2_lstm_layer'))

model.add(Dropout(dropout_prob, name='3_dropout_layer'))

model.add(Dense(classes, activation='softmax', name='softmax_layer'))

rms = optimizers.RMSprop(lr=0.001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])

In [404]:
for layer in model.layers:
    print(str(layer.name) + ': input shape: ' + str(layer.input_shape) + ' output shape: ' + str(layer.output_shape))

1_conv_layer: input shape: (None, 15, 113, 1) output shape: (None, 13, 113, 64)
2_conv_layer: input shape: (None, 13, 113, 64) output shape: (None, 11, 113, 64)
3_conv_layer: input shape: (None, 11, 113, 64) output shape: (None, 9, 113, 64)
4_conv_layer: input shape: (None, 9, 113, 64) output shape: (None, 7, 113, 64)
5_conv_layer: input shape: (None, 7, 113, 64) output shape: (None, 5, 113, 64)
6_conv_layer: input shape: (None, 5, 113, 64) output shape: (None, 3, 113, 64)
7_conv_layer: input shape: (None, 3, 113, 64) output shape: (None, 1, 113, 64)
reshape_9: input shape: (None, 1, 113, 64) output shape: (None, 1, 7232)
1_lstm_layer: input shape: (None, 1, 7232) output shape: (None, 1, 128)
2_dropout_layer: input shape: (None, 1, 128) output shape: (None, 1, 128)
2_lstm_layer: input shape: (None, 1, 128) output shape: (None, 128)
3_dropout_layer: input shape: (None, 128) output shape: (None, 128)
softmax_layer: input shape: (None, 128) output shape: (None, 18)


In [405]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1_conv_layer (Conv2D)        (None, 13, 113, 64)       256       
_________________________________________________________________
2_conv_layer (Conv2D)        (None, 11, 113, 64)       12352     
_________________________________________________________________
3_conv_layer (Conv2D)        (None, 9, 113, 64)        12352     
_________________________________________________________________
4_conv_layer (Conv2D)        (None, 7, 113, 64)        12352     
_________________________________________________________________
5_conv_layer (Conv2D)        (None, 5, 113, 64)        12352     
_________________________________________________________________
6_conv_layer (Conv2D)        (None, 3, 113, 64)        12352     
_________________________________________________________________
7_conv_layer (Conv2D)        (None, 1, 113, 64)        12352     
__________

In [406]:
batchSize = 200
train_epoches = 50
validationRatio=1.0;
model.fit(reshaped_train,train_segments_labels, validation_split=1-validationRatio,epochs=train_epoches,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(reshaped_test,test_segments_labels,verbose=1)
print(score)
model.save('taskB2_S1_CNN_LSTM_model.h5')

Epoch 1/50
22860/22860 [==============================] - 27s 1ms/step - loss: 0.9551 - acc: 0.8258
Epoch 2/50
22860/22860 [==============================] - 17s 763us/step - loss: 0.7102 - acc: 0.8344
Epoch 3/50
22860/22860 [==============================] - 18s 766us/step - loss: 0.5533 - acc: 0.8562
Epoch 4/50
22860/22860 [==============================] - 17s 757us/step - loss: 0.4873 - acc: 0.8640
Epoch 5/50
22860/22860 [==============================] - 17s 759us/step - loss: 0.4409 - acc: 0.8708
Epoch 6/50
22860/22860 [==============================] - 17s 758us/step - loss: 0.4009 - acc: 0.8805
Epoch 7/50
22860/22860 [==============================] - 17s 760us/step - loss: 0.3618 - acc: 0.8901
Epoch 8/50
22860/22860 [==============================] - 17s 761us/step - loss: 0.3336 - acc: 0.8963
Epoch 9/50
22860/22860 [==============================] - 17s 759us/step - loss: 0.3021 - acc: 0.9036
Epoch 10/50
22860/22860 [==============================] - 17s 760us/step - loss: 0.

In [363]:
predictions = model.predict(reshaped_test)

In [364]:
count = 0.0
den = 0.0
for pair in zip(predictions, test_segments_labels):
    if np.argmax(pair[0]) == np.argmax(pair[1]):
        count += 1.0
    den += 1.0

print(count / den)

0.8503805899143673


In [365]:
# F1-score measure
from sklearn.metrics import f1_score
num_classes = 18
class_predictions = []
class_true = []
tot_labels = 0.0
count = 0.0
for pair in zip(predictions, test_segments_labels):
    class_predictions.append(np.argmax(pair[0]))
    class_true.append(np.argmax(pair[1]))
    if np.argmax(pair[0]) == np.argmax(pair[1]):
        count += 1.0
    tot_labels += 1.0
    
print('Standard accuracy is ' + str(count/tot_labels))    

unique, counts = np.unique(class_true, return_counts=True)
counted_labels = dict(zip(unique, counts))
f1_scores = f1_score(class_predictions, class_true, average=None)

tot_f1_score = 0.0
weights_sum = 0.0
for i in range(num_classes):
    labels_class_i = counted_labels[i]
    weight_i = labels_class_i / tot_labels
    weights_sum += weight_i
    tot_f1_score += f1_scores[i]*weight_i
    print(str(i) + ' ' + str(weight_i) + ' ' + str(f1_scores[i]))

    
print('The weigths sum is ' + str(weights_sum))
print('The computed f1-score is {}'.format(tot_f1_score))
print('The f1-score with sklearn function is {}'.format(f1_score(class_true, class_predictions, average='weighted')))

Standard accuracy is 0.8503805899143673
0 0.763796384396 0.921625544267
1 0.0149857278782 0.774193548387
2 0.0128449096099 0.760416666667
3 0.0132017126546 0.766355140187
4 0.0105851569933 0.747252747253
5 0.0140342530923 0.461538461538
6 0.0165318744053 0.489795918367
7 0.00475737392959 0.136363636364
8 0.00701712654615 0.32
9 0.00273549000951 0.19512195122
10 0.00333016175071 0.409638554217
11 0.00463843958135 0.0
12 0.00701712654615 0.0606060606061
13 0.00749286393911 0.594202898551
14 0.00547098001903 0.518518518519
15 0.00558991436727 0.350877192982
16 0.0971693625119 0.483437779767
17 0.00880114176974 0.464646464646
The weigths sum is 1.0
The computed f1-score is 0.8234378060146824
The f1-score with sklearn function is 0.8234378060146825


In [102]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_49 (Conv1D)           (None, 11, 64)            36224     
_________________________________________________________________
conv1d_50 (Conv1D)           (None, 7, 64)             20544     
_________________________________________________________________
conv1d_51 (Conv1D)           (None, 3, 64)             20544     
_________________________________________________________________
conv1d_52 (Conv1D)           (None, 3, 64)             20544     
_________________________________________________________________
dropout_28 (Dropout)         (None, 3, 64)             0         
_________________________________________________________________
cu_dnnlstm_19 (CuDNNLSTM)    (None, 3, 128)            99328     
_________________________________________________________________
dropout_29 (Dropout)         (None, 3, 128)            0         
__________

In [38]:
pred_df = pd.DataFrame(predictions)
pred_df.to_csv('preds_test.csv', header=False, index=False)

In [39]:
true_df = pd.DataFrame(testY)
true_df.to_csv('true_test.csv', header=False, index=False)